In [1]:
import pandas as pd, numpy as np

In [26]:
#Loading the datasets
grant = pd.read_csv('Grant.csv',header = None)
publication_1 = pd.read_csv('Publication_1.csv', header = None)
publication_3 = pd.read_csv('Publication_3.csv', header = None)

In [27]:
#Selecting relevant features
grant = grant.iloc[1:, :]
grant.columns = grant.iloc[0]
grant = grant.drop(grant.index[0])
publication_1 = publication_1.iloc[1:,:]
publication_1.columns = publication_1.iloc[0]
publication_1 = publication_1[['PMID','Title','Abstract','Publication Date (print)','Authors','Authors (Raw Affiliation)'
                               ,'Corresponding Authors','Research Organizations - standardized',
                               'Country of Research organization','Funder Group', 'Times cited', 'Altmetric',
                              'Sustainable Development Goals']]
publication_1 = publication_1.drop(publication_1.index[0])
publication_3 = publication_3.iloc[1:,:]
publication_3.columns = publication_3.iloc[0]
publication_3 = publication_3[['PMID','Title','Abstract','Publication Date (print)','Authors','Authors (Raw Affiliation)'
                               ,'Corresponding Authors','Research Organizations - standardized',
                               'Country of Research organization','Funder Group', 'Times cited', 'Altmetric',
                              'Sustainable Development Goals']]
publication_3 = publication_3.drop(publication_3.index[0])

In [95]:
#Merging the two publication dataset
df_diff = publication_1[~publication_1.PMID.isin(Publication_3.PMID)]
publication_df = pd.concat([publication_3, df_diff], ignore_index= True)
publication_df.shape

(6920, 13)

In [53]:
#Loading the altmetric (larger) dataset
altmetric_df = pd.read_csv('Altmetric - Research Outputs - Texas A&M University - 2022-03-12.csv',encoding='cp1252')

C:\ProgramData\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (22,24) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [54]:
#Selecting relevant features
altmetric_df = altmetric_df[['Altmetric Attention Score','Title', 'Authors at my Institution','Departments','Funder',
                             'Publication Date','PubMed ID','News mentions','Blog mentions','Policy mentions','Patent mentions',
                             'Twitter mentions','Peer review mentions','Weibo mentions',
                             'Facebook mentions','Wikipedia mentions','Google+ mentions','LinkedIn mentions',
                             'Reddit mentions','Pinterest mentions','F1000 mentions',
                            'Q&A mentions','Video mentions','Syllabi mentions','Number of Mendeley readers',
                             'Number of Dimensions citations']]
altmetric_df = altmetric_df.rename(columns = {'PubMed ID': "PMID"})

In [55]:
#Typecasting PMID to int while null values are marked 0
altmetric_df['PMID'] = altmetric_df['PMID'].fillna(0)
altmetric_df['PMID'] = altmetric_df['PMID'].astype(int)

In [97]:
#Typecasting PMID to int and null values are marked 1
publication_df['PMID'] = publication_df['PMID'].fillna(1).astype('int')


In [159]:
#Combining the dataframes, taking care of similar columns and null values by taking a union
combined_df = pd.merge(altmetric_df, publication_df, on = ['PMID'], how = 'outer').set_index('PMID')
combined_df['Publication Date (print)'].fillna(combined_df['Publication Date'],inplace = True)
combined_df['Publication Date'].fillna(combined_df['Publication Date (print)'], inplace = True)
combined_df['Title_y'].fillna(combined_df['Title_x'],inplace= True)
combined_df['Title_x'].fillna(combined_df['Title_y'],inplace= True)
combined_df['Altmetric Attention Score'].fillna(combined_df['Altmetric'],inplace = True)
combined_df['Altmetric'].fillna(combined_df['Altmetric Attention Score'],inplace = True)
combined_df = combined_df[combined_df['Publication Date'].notna()]
combined_df.pop('Title_y')
combined_df.pop('Altmetric')
combined_df.pop('Publication Date (print)')
combined_df['Title'] = combined_df['Title_x']
combined_df.pop('Title_x')
combined_df

,Altmetric Attention Score,Authors at my Institution,Departments,Funder,Publication Date,News mentions,Blog mentions,Policy mentions,Patent mentions,Twitter mentions,...,Abstract,Authors,Authors (Raw Affiliation),Corresponding Authors,Research Organizations - standardized,Country of Research organization,Funder Group,Times cited,Sustainable Development Goals,Title
PMID,,,,,,,,,,,,,,,,,,,,,
32527856,21128,"ZHANG, Renyi",Atmospheric Sciences; Texas A&M; College of Ge...,Welch Foundation,11/06/2020,719.0,47.0,3.0,1.0,37528.0,...,Various mitigation measures have been implemen...,"Zhang, Renyi; Li, Yixin; Zhang, Annie L; Wang,...","Zhang, Renyi (Department of Atmospheric Scienc...","Zhang, Renyi (Texas A&M University; Texas A&M ...",Texas A&M University; California Institute of ...,United States; United States; United States; U...,NaN,652,NaN,Identifying airborne transmission as the domin...
28864332,7685,"KUMAR, Rajesh",Texas A&M; Medical Physiology; College of Medi...,Canadian Institutes of Health Research; Colcie...,11/1/2017,315.0,35.0,3.0,0.0,12139.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Associations of fats and carbohydrate intake w...
30518902,5670,"XU, Yangyang",Atmospheric Sciences; College of Geosciences; ...,NaN,12/5/2018,95.0,17.0,0.0,0.0,9548.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Global warming will happen faster than we think
31569235,4504,"STOVER, Patrick J; JOHNSTON, Bradley C",College of Agriculture and Life Sciences; Epid...,National Institute of Environmental Health Sci...,10/1/2019,510.0,47.0,0.0,0.0,3211.0,...,This article has been corrected. The original ...,"Johnston, Bradley C; Zeraatkar, Dena; Han, Mi ...","Johnston, Bradley C (Dalhousie University, Hal...",NaN,Hospital of the Holy Cross and Saint Paul; Net...,Spain; Netherlands; Canada; Canada; South Kore...,ICRP - International Cancer Research Partnersh...,126,NaN,Unprocessed Red Meat and Processed Meat Consum...
26315443,4156,"DONNELLAN, Michael B; THOMAS, Shawna L",Texas A&M; Computer Science and Engineering; I...,Laura and John Arnold Foundation; Medical Rese...,8/28/2015,364.0,238.0,6.0,0.0,2402.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Estimating the reproducibility of psychologica...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8447477,NaN,NaN,NaN,NaN,1993-02-01,NaN,NaN,NaN,NaN,NaN,...,The influence of flow on endothelial intracell...,"Falcone, J. C.; Kuo, L.; Meininger, G. A.","Falcone, J. C. (Department of Medical Physiolo...",NaN,Texas A&M University,United States,ICRP - International Cancer Research Partnersh...,124,NaN,Endothelial cell calcium increases during flow...
11287318,NaN,NaN,NaN,NaN,2001-05-01,NaN,NaN,NaN,NaN,NaN,...,Ras proteins are critical regulators of cell f...,"Collett, Esther D.; Davidson, Laurie A.; Fan, ...","Collett, Esther D. (Molecular and Cell Biology...",NaN,Texas A&M University,United States,ICRP - International Cancer Research Partnersh...,124,NaN,n-6 and n-3 polyunsaturated fatty acids differ...
18354641,3,NaN,NaN,NaN,2000-12-01,NaN,NaN,NaN,NaN,NaN,...,Mueller matrices provide a complete characteri...,"Jiao, S; Yao, G; Wang, L V","Jiao, S (Optical Imaging Laboratory, Biomedica...",NaN,Texas A&M University,United States,NaN,124,NaN,Depth-resolved two-dimensional stokes vectors ...


In [161]:
#Saving the file as csv
combined_df.to_csv('combined_df.csv')